In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [2]:
NDE = 10
NLJ = 10
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [3]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [4]:
# Parameters
ffpkl = "pc_pbe0dz_clusters.ff.pkl"


#  LJ fit


In [5]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


FF: LJ water_cluster ELEC harmonic intE (jax_coloumb: True)

In [6]:
# _c.targets

In [7]:
fit_repeat(_c, 
           NLJ, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:175: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [8]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
3,29.763702,401,705,0,True,Optimization terminated successfully.,"[0.06636429338741104, 1.7453106012283122, 0.49...","([[0.06636429338741104, 1.7453106012283122, 0...."
6,29.765310,383,674,0,True,Optimization terminated successfully.,"[0.053350764603618595, 1.7434291775315645, 0.5...","([[0.053350764603618595, 1.7434291775315645, 0..."
0,34.610264,574,1018,0,True,Optimization terminated successfully.,"[1.2387412754363547, 1.236692848557819, 0.0213...","([[1.2387412754363547, 1.236692848557819, 0.02..."
1,34.619930,201,404,0,True,Optimization terminated successfully.,"[1.284626066505722, 1.1666920045831595, 0.0131...","([[1.284626066505722, 1.1666920045831595, 0.01..."
8,34.638767,193,352,0,True,Optimization terminated successfully.,"[1.3291337175285167, 1.114732531326755, 0.0083...","([[1.3291337175285167, 1.114732531326755, 0.00..."
7,34.719040,255,498,0,True,Optimization terminated successfully.,"[1.4907276117136883, 0.9535757561670598, 0.002...","([[1.4907276117136883, 0.9535757561670598, 0.0..."
5,34.720638,190,373,0,True,Optimization terminated successfully.,"[1.494531138825348, 0.9500064642656461, 0.0019...","([[1.494531138825348, 0.9500064642656461, 0.00..."
2,35.805412,187,350,0,True,Optimization terminated successfully.,"[0.05, 1.6657358164445832, 0.001, 0.5]","([[0.05, 1.6657358164445832, 0.001, 0.5], [0.0..."
4,57.004795,182,339,0,True,Optimization terminated successfully.,"[1.7670454678908014, 0.05000000000000002, 0.00...","([[1.7670454678908014, 0.05000000000000002, 0...."
9,129.745026,53,147,0,True,Optimization terminated successfully.,"[0.05, 0.05, 0.1431469696030614, 0.5]","([[0.05, 0.05, 0.1431469696030614, 0.5], [0.05..."


#  DE fit


In [9]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [10]:
fit_repeat(_c, 
           NDE, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:175: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [11]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
8,30.051018,506,891,0,True,Optimization terminated successfully.,"[0.3422224239657698, 1.7162941194368841, 0.001...","([[0.3422224239657698, 1.7162941194368841, 0.0..."
6,32.356266,926,1515,0,True,Optimization terminated successfully.,"[0.3405504536710282, 1.8592562236353292, 0.011...","([[0.3405504536710282, 1.8592562236353292, 0.0..."
2,34.027721,1003,1598,0,True,Optimization terminated successfully.,"[1.3062915368423942, 1.756543627563648, 0.0216...","([[1.3062915368423942, 1.756543627563648, 0.02..."
3,34.420235,589,982,0,True,Optimization terminated successfully.,"[1.7110393505162946, 0.9160429546842916, 0.001...","([[1.7110393505162946, 0.9160429546842916, 0.0..."
5,34.933304,595,1029,0,True,Optimization terminated successfully.,"[1.392385896337885, 0.8516252549861464, 0.0010...","([[1.392385896337885, 0.8516252549861464, 0.00..."
1,36.858353,656,1091,0,True,Optimization terminated successfully.,"[1.222747329881375, 2.460079547197252, 0.13860...","([[1.222747329881375, 2.460079547197252, 0.138..."
4,42.223080,795,1334,0,True,Optimization terminated successfully.,"[2.470524261923879, 0.05, 0.009134268238258402...","([[2.470524261923879, 0.05, 0.0091342682382584..."
0,129.745026,63,239,0,True,Optimization terminated successfully.,"[0.05, 0.42405140031149546, 0.4946834966510481...","([[0.05, 0.42405140031149546, 0.49468349665104..."
7,129.745026,41,201,0,True,Optimization terminated successfully.,"[0.13267816582383452, 0.07163941108371863, 0.2...","([[0.13267816582383452, 0.07163941108371863, 0..."
9,129.745026,54,219,0,True,Optimization terminated successfully.,"[0.4117324170916983, 0.05, 0.00170589193014417...","([[0.4117324170916983, 0.05, 0.001705891930144..."
